<a href="https://colab.research.google.com/github/fabrizioaymone/machinelearning/blob/main/LGBM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
import os, sys
drive.mount("/content/drive")
if not os.path.exists("/content/datasets"):
  os.symlink("/content/drive/MyDrive/datasets/H&M", "/content/datasets")
sys.path.append("/content/drive/MyDrive/env")
!chmod 777 -R /content/drive/MyDrive/env/pyspark

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import pyspark.pandas as ps
import numpy as np

users = ps.read_csv("/content/datasets/customers.csv")
items = ps.read_csv("/content/datasets/articles.csv")
URM = ps.read_csv("/content/datasets/transactions_train.csv", dtype={'t_dat': np.datetime64})


/content/drive/MyDrive/env/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.



In [54]:
customers = users['customer_id']


#DATA PREPROCESSING

##USERS

REMOVING NAN

In [55]:
users.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [56]:
(users.isna().sum()/len(users)*100).plot.barh()

In [57]:
users = users.drop(['Active', 'FN'], axis=1)

In [58]:
users.age.fillna(users.age.mode()[0].astype('object'))

0      49
1      25
2      24
3      54
4      52
5      21
6      20
7      32
8      20
9      20
10     29
11     31
12     49
13     56
14     29
15     54
16     75
17     49
18     49
19     41
20     27
21     30
22     48
23     35
24     22
25     40
26     31
27     25
28     38
29     24
30     45
31     68
32     55
33     19
34     55
35     48
36     20
37     60
38     44
39     40
40     21
41     25
42     26
43     26
44     28
45     24
46     53
47     25
48     33
49     60
50     17
51     31
52     23
53     52
54     21
55     55
56     48
57     30
58     19
59     22
60     51
61     18
62     25
63     49
64     48
65     20
66     34
67     57
68     25
69     47
70     56
71     25
72     24
73     19
74     48
75     25
76     28
77     25
78     30
79     20
80     27
81     51
82     23
83     22
84     70
85     25
86     33
87     25
88     33
89     21
90     50
91     51
92     32
93     63
94     49
95     58
96     20
97     20
98     18
99     21


In [62]:
import pandas as pd
users.fashion_news_frequency.loc[(users.fashion_news_frequency == 'None') | (pd.isna(users.fashion_news_frequency))] = 'NONE'

In [63]:
users.fashion_news_frequency.value_counts(dropna=False).plot.barh()

In [61]:
users.head(20)

,customer_id,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,None,None,NaN,2c29ae653a9282cce4151bd87643c907644e09541abc28...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,ACTIVE,NONE,20.0,fe7b8e2b3fafb89ca90db17ffeeae0fd29b795d803f749...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,ACTIVE,Regularly,32.0,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...


In [65]:
users.club_member_status.fillna('LEFT CLUB')

0          ACTIVE
1          ACTIVE
2          ACTIVE
3          ACTIVE
4          ACTIVE
5       LEFT CLUB
6          ACTIVE
7          ACTIVE
8          ACTIVE
9          ACTIVE
10         ACTIVE
11         ACTIVE
12         ACTIVE
13         ACTIVE
14         ACTIVE
15         ACTIVE
16         ACTIVE
17         ACTIVE
18         ACTIVE
19         ACTIVE
20         ACTIVE
21         ACTIVE
22         ACTIVE
23         ACTIVE
24         ACTIVE
25     PRE-CREATE
26         ACTIVE
27         ACTIVE
28         ACTIVE
29         ACTIVE
30         ACTIVE
31         ACTIVE
32         ACTIVE
33         ACTIVE
34     PRE-CREATE
35         ACTIVE
36         ACTIVE
37         ACTIVE
38         ACTIVE
39         ACTIVE
40         ACTIVE
41         ACTIVE
42         ACTIVE
43         ACTIVE
44         ACTIVE
45         ACTIVE
46         ACTIVE
47         ACTIVE
48     PRE-CREATE
49     PRE-CREATE
50         ACTIVE
51         ACTIVE
52         ACTIVE
53         ACTIVE
54         ACTIVE
55        

In [66]:
users.club_member_status.value_counts().plot.barh()

CONVERTING DTYPES FOR MEMORY OPTIMIZATION

In [67]:
#users_i_mem = users.memory_usage().sum()
#users_i_mem

PandasNotImplementedError: ignored

In [ ]:
users.memory_usage()

In [ ]:
users.dtypes

In [ ]:
users.head()

In [ ]:
users = users.astype({'customer_id': 'string', 'club_member_status': 'category', 'fashion_news_frequency': 'category', 'age': 'uint8', 'postal_code': 'string'})

In [ ]:
users.dtypes

In [ ]:
users_f_mem = users.memory_usage().sum()
users_f_mem

In [ ]:
#MEMORY SAVINGS
memory_savings = (users_i_mem-users_f_mem)/users_i_mem*100
print(('MEMORY SAVINGS: {:.2f}%').format(memory_savings))

##ITEMS

REMOVING NAN

In [ ]:
items.head()

In [ ]:
(items.isna().sum()/len(items)*100)

In [ ]:
items.detail_desc.nunique()

TO DO:

* convert detail_desc using npl  


In [ ]:
items = items.drop('detail_desc', axis=1)

In [ ]:
items.drop(["product_code", 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no', 'index_group_name'], axis=1, inplace=True) # delete redundant categories

In [ ]:
items.head()

MEMORY OPTIMIZATION

In [ ]:
items.head()

In [ ]:
items_i_mem = items.memory_usage().sum()
items_i_mem

In [ ]:
items.dtypes

In [ ]:
items = items.astype({'article_id': 'uint32',\
              'prod_name': 'string',\
              'product_type_name': 'category',\
              'product_group_name': 'category',\
              'graphical_appearance_name': 'category',\
              'colour_group_name': 'category',\
              'perceived_colour_value_name': 'category',\
              'perceived_colour_master_name': 'category',\
              'department_name': 'category',\
              'index_name': 'category',\
              'section_name': 'category',\
              'garment_group_name': 'category'})

In [ ]:
items.prod_name.memory_usage()

In [ ]:
items.prod_name.astype('category').memory_usage() #in this case is not convenient to change the dtype from string to category

In [ ]:
items_f_mem = items.memory_usage().sum()
memory_savings = (items_i_mem-items_f_mem)/items_i_mem*100
print(('MEMORY SAVINGS: {:.2f}%').format(memory_savings))

##URM

In [ ]:
URM.head()

In [ ]:
URM.isna().sum()

In [ ]:
URM.drop(['price', 'sales_channel_id'], axis=1, inplace=True)

MEMORY OPTIMIZATION

In [ ]:
URM

In [ ]:
URM_i_mem = URM.memory_usage().sum()
URM_i_mem

In [ ]:
URM = URM.astype({'customer_id': 'string', 'article_id': 'uint'})

In [ ]:
URM_f_mem = URM.memory_usage().sum()
URM_f_mem

In [ ]:
#MEMORY SAVINGS
memory_savings = (URM_i_mem-URM_f_mem)/URM_i_mem*100
print(('MEMORY SAVINGS: {:.2f}%').format(memory_savings))

#FEATURE ENGINEERING

TIME

In [ ]:
min_time = URM.t_dat.min()
max_time = URM.t_dat.max()
min_time, max_time

In [ ]:
total_weeks = (URM.t_dat.max()-URM.t_dat.min())/pd.Timedelta(weeks=1)
print(f"the total number of weeks is :   {total_weeks}")

In [ ]:
URM.t_dat.min().dayofweek, URM.t_dat.max().dayofweek # this explains why the total number of weeks is not an integer

In [ ]:

#WEEK
URM['week'] = ((URM['t_dat'].max()-URM['t_dat'])/pd.Timedelta(1, 'W')).astype('uint8')+1

The [1st place solution](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/324070) uses a maximum of 25 weeks 

They have 5 different train set where they train 5 different models: \\
7  week -> 1 week train 1 week -> now valid \\
12  week -> 1 week train 1 week -> now valid \\
16  week -> 1 week train 1 week -> now valid \\
20  week -> 1 week train 1 week -> now valid \\
25  week -> 1 week train 1 week -> now valid \\

In [ ]:
max_time-pd.Timedelta(25, 'W')

In [ ]:
# MEMORY WILL CRASH IN THE FOLLOWING COMMAND SO WE HAVE TO REDUCE OUR DATAFRAME
# keep the first 6 weeks + 1
URM = URM[URM['t_dat'] > max_time-pd.Timedelta(7, 'W')]
URM.drop('t_dat', axis=1, inplace=True)

We will consider last month as 4 week period preceeding the week in exam

In [ ]:
#MONTH
#[URM['month'] = (URM['t_dat'].max().to_period('M') - URM['t_dat'].dt.to_period('M')).apply(lambda x: x.n)

In [ ]:
#YEAR
#URM['year'] = URM['t_dat'].year.max() - URM['t_dat'].dt.year

In [ ]:
URM = URM.sort_values('week')
URM['y'] = 1
URM

# TOP 12 FROM LAST WEEK

In [ ]:
transactions = URM[['customer_id', 'article_id','week']]

In [ ]:
transactions.groupby('week')['week'].count()

In [ ]:
transactions_rank = transactions.groupby(['week', 'article_id']).count().sort_values(['week', 'customer_id'], ascending=[True, False]).rename(columns={'customer_id': 'count'})


In [ ]:
transactions_rank['rank'] = transactions_rank.groupby(level=0).cumcount()+1

In [ ]:
transactions_rank

In [ ]:
transactions_rank = transactions_rank[transactions_rank['rank']<=12].drop('count', axis=1)

In [ ]:
transactions_rank.reset_index(inplace=True)

In [ ]:
transactions_rank['week'] = transactions_rank['week']-1
transactions_rank

In [ ]:
top_12 =pd.merge(transactions_rank.drop('rank', axis=1), customers, how='cross')

In [ ]:
del transactions, transactions_rank

#LAST WEEK PURCHASE 

In [ ]:
lw_purchase = URM.drop('y',axis=1)
lw_purchase['week'] = lw_purchase['week']-1

In [ ]:
lw_purchase

In [ ]:
lw_purchase['rank'] = lw_purchase.groupby(['customer_id', 'week']).cumcount()
print(len(lw_purchase))
lw_purchase = lw_purchase[lw_purchase['rank']<1].drop('rank',axis=1)
print(len(lw_purchase))

#FINAL URM

In [ ]:
final = URM.merge(top_12, on=['week', 'customer_id', 'article_id'],  how='outer').fillna(0)
del URM
final

In [ ]:
final.dtypes

In [ ]:
lw_purchase.dtypes

In [ ]:
final = final.astype({'y':'uint8', 'week': 'uint8', 'customer_id': 'string'})

In [ ]:
''' final = final.merge(lw_purchase, on=['week', 'customer_id', 'article_id'],  how='outer').fillna(0)
final #keeps crashing because of ram spike '''

In [ ]:
final

In [ ]:
final.merge(customers, on='customer_id')

MODEL TRAINING

In [ ]:
import lightgbm as lgbm


In [ ]:
final_train = final[final['week']>1]
final_val = final[final['week']]